In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import warnings
import json
warnings.filterwarnings('ignore')

print("✅ Libraries imported")
print(f"TensorFlow version: {tf.__version__}")

✅ Libraries imported
TensorFlow version: 2.20.0


In [2]:
# ======================================================
# 🔹 LOAD AND MERGE MAIN DATASETS (RUN THIS FIRST)
# ======================================================

print("🔹 LOADING ALL MAIN DATASETS...")

# --- 1. Actual Sugar Data ---
print("📈 Loading Actual Sugar data...")
sugar_df = pd.read_csv(r"C:\Users\ramhya.kathirayson\Downloads\sugardata.csv")
sugar_df['Date'] = pd.to_datetime(sugar_df['Date'], errors='coerce')
sugar_df.dropna(subset=['Date'], inplace=True)
sugar_df = sugar_df[['Date', 'Value']].rename(columns={'Value': 'SUGAR'})
sugar_df['SUGAR'] = pd.to_numeric(sugar_df['SUGAR'], errors='coerce')
sugar_df.dropna(subset=['SUGAR'], inplace=True)
sugar_df.set_index('Date', inplace=True)
print(f"✅ Sugar data: {len(sugar_df)} records")

# --- 2. Crude Oil Data ---
print("🛢️ Loading Crude Oil data...")
crude_df = pd.read_excel(r"C:\Users\ramhya.kathirayson\Downloads\crudeoildata.xlsx")
crude_df['Date'] = pd.to_datetime(crude_df['Date'], errors='coerce')
crude_df = crude_df[['Date', 'Price']].rename(columns={'Price': 'Crude_Oil_Price'})
crude_df.dropna(subset=['Date', 'Crude_Oil_Price'], inplace=True)
print(f"✅ Crude Oil data: {len(crude_df)} records")

# --- 3. Exchange Rate Data ---
print("💱 Loading Exchange Rate data...")
fx = pd.read_csv(r"C:\Users\ramhya.kathirayson\Downloads\exchange_rate_to_usd.csv")
fx['date'] = pd.to_datetime(fx['date'], errors='coerce')
fx = fx[['date', 'us_dollar_to_usd', 'brazilian_real_to_usd', 'thai_baht_to_usd',
         'indian_rupee_to_usd', 'malaysian_ringgit_to_usd']]
fx.columns = ['Date', 'USD_to_USD', 'BRL_to_USD', 'THB_to_USD', 'INR_to_USD', 'MYR_to_USD']
fx = fx.ffill().bfill()
print(f"✅ Exchange Rate data: {len(fx)} records")

# --- 4. ICE NY11 Data ---
print("🧊 Loading ICE NY11 data...")

def load_daily_ice_data(ice_file_path):
    """Load and clean ICE NY11 daily data"""
    try:
        ice_df = pd.read_csv(ice_file_path)
        print("📋 ICE DataFrame columns:", ice_df.columns.tolist())
        
        if 'Date' not in ice_df.columns or 'Close' not in ice_df.columns:
            raise ValueError("❌ ICE data must have 'Date' and 'Close' columns")
        
        ice_df['Date'] = pd.to_datetime(ice_df['Date'], format='%b %d, %Y', errors='coerce')
        ice_df.dropna(subset=['Date'], inplace=True)
        
        ice_df['Close'] = pd.to_numeric(ice_df['Close'], errors='coerce')
        ice_df = ice_df.dropna(subset=['Close'])
        
        ice_clean = ice_df[['Date', 'Close']].rename(columns={'Close': 'ICE11_Price'})
        print(f"✅ ICE data loaded: {len(ice_clean)} records")
        return ice_clean

    except Exception as e:
        print(f"❌ Error loading ICE data: {e}")
        return None

ice_df = load_daily_ice_data(r"C:\Users\ramhya.kathirayson\Downloads\ICE_Data.csv")

# ======================================================
# 🔹 MERGE ALL DATASETS
# ======================================================

print("\n🔹 Merging all datasets...")
merged_main = (
    sugar_df.reset_index()
    .merge(crude_df, on='Date', how='inner')
    .merge(fx, on='Date', how='inner')
)
merged_main.set_index('Date', inplace=True)

print(f"Main data shape before ICE merge: {merged_main.shape}")

if ice_df is not None:
    ice_df_indexed = ice_df.set_index('Date')
    overlap = merged_main.index.intersection(ice_df_indexed.index)
    print(f"Overlapping dates with ICE: {len(overlap)}")

    if len(overlap) > 0:
        df = merged_main.merge(ice_df_indexed[['ICE11_Price']], left_index=True, right_index=True, how='inner')
        df['ICE11_Synthetic'] = False
        print(f"✅ Merged with real ICE data | Final shape: {df.shape}")
    else:
        # Synthetic ICE data fallback
        np.random.seed(42)
        merged_main['ICE11_Price'] = merged_main['SUGAR'] * np.random.uniform(0.95, 1.05, len(merged_main))
        merged_main['ICE11_Synthetic'] = True
        df = merged_main
        print("⚠️ No overlap — synthetic ICE data created")
else:
    df = merged_main
    df['ICE11_Synthetic'] = True
    print("⚠️ Proceeding without ICE data")

# Final cleaning
df = df.sort_index().dropna()
print(f"\n✅ Final dataset ready | Shape: {df.shape}")
print(f"Columns: {df.columns.tolist()}")
print(f"Date range: {df.index.min()} to {df.index.max()}")
print(f"SUGAR price range: ${df['SUGAR'].min():.4f} - ${df['SUGAR'].max():.4f}")

🔹 LOADING ALL MAIN DATASETS...
📈 Loading Actual Sugar data...
✅ Sugar data: 15797 records
🛢️ Loading Crude Oil data...
✅ Crude Oil data: 6549 records
💱 Loading Exchange Rate data...
✅ Exchange Rate data: 5471 records
🧊 Loading ICE NY11 data...
📋 ICE DataFrame columns: ['Date', 'Close', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6']
✅ ICE data loaded: 6427 records

🔹 Merging all datasets...
Main data shape before ICE merge: (5330, 7)
Overlapping dates with ICE: 5318
✅ Merged with real ICE data | Final shape: (5318, 9)

✅ Final dataset ready | Shape: (5318, 9)
Columns: ['SUGAR', 'Crude_Oil_Price', 'USD_to_USD', 'BRL_to_USD', 'THB_to_USD', 'INR_to_USD', 'MYR_to_USD', 'ICE11_Price', 'ICE11_Synthetic']
Date range: 2004-01-05 00:00:00 to 2025-10-07 00:00:00
SUGAR price range: $0.0622 - $0.3965


In [3]:
def prepare_lstm_data(df, target_col='SUGAR', sequence_length=60, test_size=0.2):
    """Prepare data for LSTM"""
    # Separate features and target
    feature_columns = [col for col in df.columns if col != target_col]
    X_raw = df[feature_columns].values
    y_raw = df[[target_col]].values

    # Initialize and fit scalers
    feature_scaler = MinMaxScaler()
    target_scaler = MinMaxScaler()

    X_scaled = feature_scaler.fit_transform(X_raw)
    y_scaled = target_scaler.fit_transform(y_raw)

    # Create LSTM sequences
    X, y = [], []
    for i in range(sequence_length, len(X_scaled)):
        X.append(X_scaled[i-sequence_length:i])
        y.append(y_scaled[i, 0])

    X, y = np.array(X), np.array(y)

    # Time-based split
    split_index = int(len(X) * (1 - test_size))
    X_train, X_test = X[:split_index], X[split_index:]
    y_train, y_test = y[:split_index], y[split_index:]

    print(f"✅ Data prepared:")
    print(f"  X_train: {X_train.shape}, y_train: {y_train.shape}")
    print(f"  X_test: {X_test.shape}, y_test: {y_test.shape}")
    print(f"  Features: {len(feature_columns)}")

    return (X_train, X_test, y_train, y_test, feature_scaler, target_scaler, feature_columns)

In [4]:
def create_optimized_lstm_features(df, target_col='SUGAR'):
    """Create optimized features"""
    df = df.copy()
    
    print("🔄 Creating features...")
    
    # Core price features
    df['Price_Lag_1'] = df[target_col].shift(1)
    df['Price_Lag_5'] = df[target_col].shift(5)
    df['Price_Lag_20'] = df[target_col].shift(20)
    
    # Rolling statistics
    df['Roll_Mean_7'] = df[target_col].rolling(7).mean()
    df['Roll_Std_7'] = df[target_col].rolling(7).std()
    
    # Momentum indicators
    df['Price_Change_1'] = df[target_col].pct_change(1)
    df['Price_Change_5'] = df[target_col].pct_change(5)
    
    # External features
    if 'Crude_Oil_Price' in df.columns:
        df['Oil_Change'] = df['Crude_Oil_Price'].pct_change()
    
    if 'BRL_to_USD' in df.columns:
        df['BRL_Change'] = df['BRL_to_USD'].pct_change()
    
    # ICE relationship
    if 'ICE11_Price' in df.columns and df['ICE11_Price'].nunique() > 10:
        df['Basis'] = df[target_col] - df['ICE11_Price']
    
    # Time features
    df['Month'] = df.index.month
    df['DayOfWeek'] = df.index.dayofweek
    
    # Remove NaN
    df = df.dropna()
    
    print(f"✅ Created {len([col for col in df.columns if col != target_col])} features")
    return df

In [5]:
print("🔹 APPLYING FEATURE ENGINEERING...")

# Apply feature engineering
df_enhanced = create_optimized_lstm_features(df)
print(f"✅ Enhanced dataset shape: {df_enhanced.shape}")

# FIX PRICE SCALE - Convert from cents to dollars
print("\n🔹 FIXING PRICE SCALE...")
print(f"Before scaling - SUGAR price: ${df_enhanced['SUGAR'].iloc[-1]:.4f}")

# Multiply by 100 to convert cents to dollars
df_enhanced_fixed = df_enhanced.copy()
df_enhanced_fixed['SUGAR'] = df_enhanced_fixed['SUGAR'] * 100

# Also scale other price columns if they exist
price_columns = ['ICE11_Price', 'Crude_Oil_Price']
for col in price_columns:
    if col in df_enhanced_fixed.columns:
        if df_enhanced_fixed[col].mean() < 10:  # If it looks like cents, scale it
            df_enhanced_fixed[col] = df_enhanced_fixed[col] * 100
            print(f"✅ Scaled {col} to dollars")

print(f"After scaling - SUGAR price: ${df_enhanced_fixed['SUGAR'].iloc[-1]:.2f}")
print("✅ Price scale fixed (cents → dollars)")

🔹 APPLYING FEATURE ENGINEERING...
🔄 Creating features...
✅ Created 20 features
✅ Enhanced dataset shape: (5298, 21)

🔹 FIXING PRICE SCALE...
Before scaling - SUGAR price: $0.1644
After scaling - SUGAR price: $16.44
✅ Price scale fixed (cents → dollars)


In [6]:
print("\n🔹 PREPARING LSTM DATA...")
sequence_length = 60

X_train, X_test, y_train, y_test, feature_scaler, target_scaler, feature_columns = prepare_lstm_data(
    df_enhanced_fixed,  # Use the FIXED dataset
    target_col='SUGAR', 
    sequence_length=sequence_length, 
    test_size=0.2
)

last_sequence = X_test[-1]

print(f"✅ Data preparation complete!")
print(f"   Last sequence shape: {last_sequence.shape}")
print(f"   Current price: ${df_enhanced_fixed['SUGAR'].iloc[-1]:.2f}")


🔹 PREPARING LSTM DATA...
✅ Data prepared:
  X_train: (4190, 60, 20), y_train: (4190,)
  X_test: (1048, 60, 20), y_test: (1048,)
  Features: 20
✅ Data preparation complete!
   Last sequence shape: (60, 20)
   Current price: $16.44


In [7]:
print("\n🔹 CREATING AND TRAINING LSTM MODEL...")

input_shape = (X_train.shape[1], X_train.shape[2])
print(f"Input shape: {input_shape}")

# Create model
final_model = Sequential([
    LSTM(100, return_sequences=True, input_shape=input_shape, recurrent_dropout=0.2),
    Dropout(0.2),
    LSTM(50, return_sequences=False, recurrent_dropout=0.2),
    Dropout(0.2),
    Dense(25, activation='relu'),
    Dense(1, activation='linear')
])

final_model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='mse',
    metrics=['mae', 'mape']
)

final_model.summary()

# Define callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True, verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=8, min_lr=0.0001, verbose=1)

print("🔄 Training model...")
history = final_model.fit(
    X_train, y_train,
    batch_size=32,
    epochs=50,
    validation_data=(X_test, y_test),
    callbacks=[early_stopping, reduce_lr],
    verbose=1,
    shuffle=False
)

print("✅ Model training complete!")


🔹 CREATING AND TRAINING LSTM MODEL...
Input shape: (60, 20)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 60, 100)        │        48,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 60, 100)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 50)             │        30,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 50)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 25)             │         1,275 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            26 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 79,901 (312.11 KB)

 Trainable params: 79,901 (312.11 KB)

 Non-trainable params: 0 (0.00 B)

🔄 Training model...
Epoch 1/50
131/131 ━━━━━━━━━━━━━━━━━━━━ 13s 63ms/step - loss: 0.0161 - mae: 0.0982 - mape: 38.3637 - val_loss: 0.0317 - val_mae: 0.1592 - val_mape: 35.8704 - learning_rate: 0.0010
Epoch 2/50
131/131 ━━━━━━━━━━━━━━━━━━━━ 8s 61ms/step - loss: 0.0208 - mae: 0.1104 - mape: 41.1704 - val_loss: 0.0570 - val_mae: 0.2263 - val_mape: 52.6415 - learning_rate: 0.0010
Epoch 3/50
131/131 ━━━━━━━━━━━━━━━━━━━━ 9s 69ms/step - loss: 0.0165 - mae: 0.0936 - mape: 32.9692 - val_loss: 0.0512 - val_mae: 0.2139 - val_mape: 49.6998 - learning_rate: 0.0010
Epoch 4/50
131/131 ━━━━━━━━━━━━━━━━━━━━ 9s 68ms/step - loss: 0.0228 - mae: 0.1074 - mape: 34.2685 - val_loss: 0.0414 - val_mae: 0.1913 - val_mape: 44.3250 - learning_rate: 0.0010
Epoch 5/50
131/131 ━━━━━━━━━━━━━━━━━━━━ 10s 75ms/step - loss: 0.0206 - mae: 0.0998 - mape: 30.7979 - val_loss: 0.0256 - val_mae: 0.1472 - val_mape: 33.7493 - learning_rate: 0.0010
Epoch 6/50
131/131 ━━━━━━━━━━━━━━━━━━━━ 9s 71ms/step - loss: 0.0156 - mae: 0.0827 -

In [8]:
def load_ice_futures_from_excel(excel_file_path):
    """Load ICE futures data"""
    try:
        print(f"📂 Loading ICE futures: {excel_file_path}")
        futures_df = pd.read_excel(excel_file_path)
        print("✅ Excel file loaded")
        
        # Find contract column
        if 'Contract' not in futures_df.columns:
            contract_col = None
            for col in futures_df.columns:
                if 'contract' in col.lower() or 'symbol' in col.lower():
                    contract_col = col
                    break
            if contract_col:
                futures_df = futures_df.rename(columns={contract_col: 'Contract'})
            else:
                raise ValueError("❌ No contract column found")
        
        # Find price column
        price_col = None
        for col in ['Last', 'Close', 'Settle', 'Previous', 'Price']:
            if col in futures_df.columns:
                price_col = col
                break
        if not price_col:
            raise ValueError("❌ No price column found")
        
        print(f"✅ Using price column: {price_col}")
        
        # Parse contract dates
        def parse_futures_contract(contract_str):
            try:
                if pd.isna(contract_str):
                    return pd.NaT
                contract_str = str(contract_str).strip()
                
                futures_codes = {'H': 3, 'K': 5, 'N': 7, 'V': 9, 'F': 1, 'G': 2, 
                               'J': 4, 'M': 6, 'Q': 8, 'U': 10, 'X': 11, 'Z': 12}
                
                # Extract from formats like "SBH26" or "SBH26 (Mar '26)"
                month_code = None
                year_digits = None
                
                if len(contract_str) >= 3 and contract_str.startswith('SB'):
                    month_code = contract_str[2].upper()
                    year_part = contract_str[3:]
                    year_digits = ''.join(filter(str.isdigit, year_part))[:2]
                
                if month_code and month_code in futures_codes and year_digits:
                    month = futures_codes[month_code]
                    year = 2000 + int(year_digits)
                    return pd.Timestamp(year=year, month=month, day=1) + pd.offsets.MonthEnd(1)
                return pd.NaT
            except:
                return pd.NaT
        
        futures_df['Contract_Date'] = futures_df['Contract'].apply(parse_futures_contract)
        futures_df = futures_df.dropna(subset=['Contract_Date'])
        
        # Clean prices
        def clean_futures_price(price_val):
            try:
                if pd.isna(price_val):
                    return np.nan
                if isinstance(price_val, str):
                    price_val = price_val.replace('s', '').strip()
                    price_val = ''.join(c for c in price_val if c.isdigit() or c == '.' or c == '-')
                return float(price_val)
            except:
                return np.nan
        
        futures_df['Cleaned_Price'] = futures_df[price_col].apply(clean_futures_price)
        futures_df = futures_df.dropna(subset=['Cleaned_Price'])
        
        # Create final dataframe
        futures_clean = futures_df[['Contract_Date', 'Cleaned_Price', 'Contract']].rename(
            columns={'Cleaned_Price': 'ICE_Futures_Price', 'Contract_Date': 'Date'}
        ).sort_values('Date')
        
        print(f"✅ ICE Futures loaded: {len(futures_clean)} contracts")
        return futures_clean

    except Exception as e:
        print(f"❌ Error: {e}")
        return None

In [9]:
# ======================================================
# 🔹 LOAD ICE FUTURES DATA
# ======================================================

print("\n📊 LOADING ICE FUTURES DATA...")
ice_futures = load_ice_futures_from_excel(r"C:\Users\ramhya.kathirayson\Downloads\Ice_futures.xlsx")

if ice_futures is None or len(ice_futures) == 0:
    print("⚠️ Creating sample ICE futures data...")
    current_price = df_enhanced_fixed['SUGAR'].iloc[-1]
    futures_data = [
        ('SBH26', '2026-03-31', current_price * 0.98),
        ('SBK26', '2026-05-31', current_price * 0.97),
        ('SBN26', '2026-07-31', current_price * 0.96),
        ('SBV26', '2026-09-30', current_price * 0.95),
        ('SBH27', '2027-03-31', current_price * 0.94),
        ('SBK27', '2027-05-31', current_price * 0.93),
    ]
    ice_futures = pd.DataFrame([
        {'Contract': contract, 'Date': pd.to_datetime(date), 'ICE_Futures_Price': round(price, 2)}
        for contract, date, price in futures_data
    ])

print(f"📊 ICE Futures contracts: {len(ice_futures)}")
print(f"💰 ICE Futures price range: ${ice_futures['ICE_Futures_Price'].min():.2f} - ${ice_futures['ICE_Futures_Price'].max():.2f}")


📊 LOADING ICE FUTURES DATA...
📂 Loading ICE futures: C:\Users\ramhya.kathirayson\Downloads\Ice_futures.xlsx
✅ Excel file loaded
✅ Using price column: Last
✅ ICE Futures loaded: 11 contracts
📊 ICE Futures contracts: 11
💰 ICE Futures price range: $14.36 - $15.32


In [11]:
# ======================================================
# 🔹 FIXED FORECASTING FUNCTION (CORRECTED TIMEDELTA ISSUE)
# ======================================================

def generate_3month_forecasts_complete(model, last_sequence, target_scaler, start_year=2026, end_year=2027):
    """Generate forecasts for ALL quarters with corrected date handling"""
    
    # Define ALL quarterly dates
    forecast_dates = []
    current_date = pd.Timestamp(f'{start_year}-03-31')
    end_date = pd.Timestamp(f'{end_year}-12-31')
    
    while current_date <= end_date:
        forecast_dates.append(current_date)
        current_date += pd.DateOffset(months=3)
        if current_date.month not in [3, 6, 9, 12]:
            current_date = current_date + pd.offsets.QuarterEnd()
    
    print(f"📅 Generating forecasts for ALL {len(forecast_dates)} quarters:")
    
    forecasts = []
    last_actual_date = df_enhanced_fixed.index[-1]
    last_actual_price = df_enhanced_fixed['SUGAR'].iloc[-1]
    
    print(f"🔮 Last actual data: {last_actual_date.strftime('%Y-%m-%d')} at ${last_actual_price:.2f}")
    
    # Get base prediction
    base_prediction_scaled = model.predict(last_sequence.reshape(1, *last_sequence.shape), verbose=0)[0, 0]
    base_prediction = target_scaler.inverse_transform(np.array([[base_prediction_scaled]]))[0, 0]
    
    print(f"🔮 Base prediction: ${base_prediction:.2f}")
    
    for target_date in forecast_dates:
        # FIX: Convert Timedelta to integer days
        days_ahead = (target_date - last_actual_date).days  # This gives integer days
        
        try:
            # IMPROVED: Use more realistic time-based adjustment
            horizon_factor = days_ahead / 365.0
            
            # IMPROVED: Use ICE futures as guidance when available
            ice_adjustment = 0.0
            if ice_futures is not None:
                # Find closest ICE futures contract
                future_diffs = (ice_futures['Date'] - target_date).abs()
                if future_diffs.min().days <= 45:  # FIX: Use .days for comparison
                    closest_idx = future_diffs.idxmin()
                    closest_future = ice_futures.loc[closest_idx]
                    ice_price = closest_future['ICE_Futures_Price']
                    # Blend model prediction with ICE futures (30% weight to ICE)
                    ice_adjustment = (ice_price - base_prediction) * 0.3
            
            # IMPROVED: More realistic random variation
            random_variation = np.random.normal(0, horizon_factor * 0.08)  # Reduced from 0.1 to 0.08
            
            predicted_price = base_prediction * (1 + random_variation) + ice_adjustment
            
            # Ensure price doesn't go negative
            predicted_price = max(predicted_price, 0.01)
            
            # Calculate confidence
            confidence = max(0.7 - horizon_factor * 0.4, 0.3)
            
            forecast_data = {
                'date': target_date,
                'forecast_price': float(predicted_price),
                'days_ahead': days_ahead,
                'confidence': float(confidence),
                'quarter': f"Q{(target_date.month-1)//3 + 1} {target_date.year}"
            }
            
            forecasts.append(forecast_data)
            
            ice_info = f" (ICE guided)" if abs(ice_adjustment) > 0.1 else ""
            print(f"   {target_date.strftime('%b %Y')}: ${predicted_price:.2f} (+{days_ahead} days, {confidence:.1%} confidence{ice_info})")
            
        except Exception as e:
            print(f"❌ Error forecasting {target_date.strftime('%b %Y')}: {e}")
            continue
    
    return forecasts

# Generate complete forecasts
print("\n🎯 GENERATING COMPLETE 2026-2027 QUARTERLY FORECASTS...")
quarterly_forecasts_complete = generate_3month_forecasts_complete(
    final_model, 
    last_sequence, 
    target_scaler,
    start_year=2026, 
    end_year=2027
)

print(f"✅ Generated {len(quarterly_forecasts_complete)} complete quarterly forecasts")


🎯 GENERATING COMPLETE 2026-2027 QUARTERLY FORECASTS...
📅 Generating forecasts for ALL 8 quarters:
🔮 Last actual data: 2025-10-07 at $16.44
🔮 Base prediction: $15.62
   Mar 2026: $15.43 (+175 days, 50.8% confidence (ICE guided))
   Jun 2026: $15.30 (+266 days, 40.8% confidence (ICE guided))
   Sep 2026: $14.17 (+358 days, 30.8% confidence (ICE guided))
   Dec 2026: $17.45 (+449 days, 30.0% confidence)
   Mar 2027: $15.40 (+539 days, 30.0% confidence (ICE guided))
   Jun 2027: $13.04 (+631 days, 30.0% confidence (ICE guided))
   Sep 2027: $16.69 (+723 days, 30.0% confidence (ICE guided))
   Dec 2027: $12.66 (+814 days, 30.0% confidence)
✅ Generated 8 complete quarterly forecasts


Model improvement strategies

In [13]:
# ======================================================
# 🔹 MODEL IMPROVEMENT STRATEGIES (FIXED)
# ======================================================

print("\n🔧 IMPROVING MODEL ACCURACY...")

def generate_ice_guided_forecasts(model, last_sequence, target_scaler, start_year=2026, end_year=2027):
    """Generate forecasts using ICE futures as guidance with corrected date handling"""
    
    forecast_dates = []
    current_date = pd.Timestamp(f'{start_year}-03-31')
    end_date = pd.Timestamp(f'{end_year}-12-31')
    
    while current_date <= end_date:
        forecast_dates.append(current_date)
        current_date += pd.DateOffset(months=3)
        if current_date.month not in [3, 6, 9, 12]:
            current_date = current_date + pd.offsets.QuarterEnd()
    
    print(f"📅 Generating ICE-guided forecasts for {len(forecast_dates)} quarters:")
    
    forecasts = []
    last_actual_date = df_enhanced_fixed.index[-1]
    last_actual_price = df_enhanced_fixed['SUGAR'].iloc[-1]
    
    # Get base model prediction
    base_prediction_scaled = model.predict(last_sequence.reshape(1, *last_sequence.shape), verbose=0)[0, 0]
    base_prediction = target_scaler.inverse_transform(np.array([[base_prediction_scaled]]))[0, 0]
    
    print(f"🔮 Base model prediction: ${base_prediction:.2f}")
    
    for target_date in forecast_dates:
        # FIX: Convert Timedelta to integer days
        days_ahead = (target_date - last_actual_date).days
        
        try:
            # Find corresponding ICE futures price
            ice_guidance = None
            if ice_futures is not None:
                future_diffs = (ice_futures['Date'] - target_date).abs()
                if future_diffs.min().days <= 45:  # FIX: Use .days for comparison
                    closest_idx = future_diffs.idxmin()
                    closest_future = ice_futures.loc[closest_idx]
                    ice_guidance = closest_future['ICE_Futures_Price']
            
            # BLEND MODEL WITH ICE FUTURES
            if ice_guidance is not None:
                # Use weighted average: 60% model + 40% ICE futures
                blend_ratio = 0.6
                predicted_price = (base_prediction * blend_ratio) + (ice_guidance * (1 - blend_ratio))
                method = "BLENDED"
            else:
                # Fallback to model with time-based adjustment
                horizon_factor = days_ahead / 365.0
                random_variation = np.random.normal(0, horizon_factor * 0.05)  # Reduced variation
                predicted_price = base_prediction * (1 + random_variation)
                method = "MODEL_ONLY"
            
            predicted_price = max(predicted_price, 0.01)
            
            # Confidence based on method
            if method == "BLENDED":
                confidence = max(0.8 - (days_ahead / 365) * 0.3, 0.4)  # Higher confidence for blended
            else:
                confidence = max(0.7 - (days_ahead / 365) * 0.4, 0.3)
            
            forecast_data = {
                'date': target_date,
                'forecast_price': float(predicted_price),
                'days_ahead': days_ahead,
                'confidence': float(confidence),
                'quarter': f"Q{(target_date.month-1)//3 + 1} {target_date.year}",
                'method': method,
                'ice_guidance_used': ice_guidance is not None
            }
            
            forecasts.append(forecast_data)
            
            method_info = f" ({method})" if method == "BLENDED" else ""
            print(f"   {target_date.strftime('%b %Y')}: ${predicted_price:.2f} (+{days_ahead} days, {confidence:.1%}{method_info})")
            
        except Exception as e:
            print(f"❌ Error forecasting {target_date.strftime('%b %Y')}: {e}")
            continue
    
    return forecasts

# Generate ICE-guided forecasts
print("\n🎯 GENERATING ICE-GUIDED FORECASTS...")
quarterly_forecasts_improved = generate_ice_guided_forecasts(
    final_model, 
    last_sequence, 
    target_scaler,
    start_year=2026, 
    end_year=2027
)

print(f"✅ Generated {len(quarterly_forecasts_improved)} improved forecasts")


🔧 IMPROVING MODEL ACCURACY...

🎯 GENERATING ICE-GUIDED FORECASTS...
📅 Generating ICE-guided forecasts for 8 quarters:
🔮 Base model prediction: $15.62
   Mar 2026: $15.36 (+175 days, 65.6% (BLENDED))
   Jun 2026: $15.16 (+266 days, 58.1% (BLENDED))
   Sep 2026: $15.20 (+358 days, 50.6% (BLENDED))
   Dec 2026: $15.89 (+449 days, 30.0%)
   Mar 2027: $15.41 (+539 days, 40.0% (BLENDED))
   Jun 2027: $15.30 (+631 days, 40.0% (BLENDED))
   Sep 2027: $15.32 (+723 days, 40.0% (BLENDED))
   Dec 2027: $18.15 (+814 days, 30.0%)
✅ Generated 8 improved forecasts


In [14]:
def compare_with_ice_futures(quarterly_forecasts, ice_futures):
    """Compare forecasts with ICE futures"""
    
    if ice_futures is None or len(ice_futures) == 0:
        print("❌ No ICE futures data")
        return None
    
    comparison_results = []
    
    for forecast in quarterly_forecasts:
        forecast_date = forecast['date']
        
        # Find closest ICE futures contract
        future_diffs = (ice_futures['Date'] - forecast_date).abs()
        closest_idx = future_diffs.idxmin()
        closest_future = ice_futures.loc[closest_idx]
        
        days_diff = (closest_future['Date'] - forecast_date).days
        futures_price = closest_future['ICE_Futures_Price']
        forecast_price = forecast['forecast_price']
        
        # Compare if dates are close
        if abs(days_diff) <= 45:
            price_diff = forecast_price - futures_price
            price_diff_pct = (price_diff / futures_price) * 100
            
            comparison = {
                'forecast_date': forecast_date,
                'futures_date': closest_future['Date'],
                'days_difference': days_diff,
                'model_forecast': forecast_price,
                'ice_futures': futures_price,
                'price_difference': price_diff,
                'price_difference_pct': price_diff_pct,
                'model_confidence': forecast['confidence'],
                'comparison_quarter': forecast['quarter']
            }
            
            comparison_results.append(comparison)
            
            direction = "ABOVE" if price_diff_pct > 0 else "BELOW"
            print(f"🔍 {forecast['quarter']}: Model ${forecast_price:.2f} vs ICE ${futures_price:.2f} ({abs(price_diff_pct):.1f}% {direction})")
    
    return pd.DataFrame(comparison_results)

Compare Improved forecasts

In [15]:
# ======================================================
# 🔹 COMPARE IMPROVED FORECASTS
# ======================================================

print("\n📊 COMPARING IMPROVED FORECASTS WITH ICE FUTURES...")

futures_comparison_improved = compare_with_ice_futures(quarterly_forecasts_improved, ice_futures)

if futures_comparison_improved is not None:
    print(f"✅ Compared {len(futures_comparison_improved)} improved forecasts")
    
    avg_diff_improved = futures_comparison_improved['price_difference_pct'].mean()
    max_diff_improved = futures_comparison_improved['price_difference_pct'].abs().max()
    
    print(f"\n📈 IMPROVEMENT SUMMARY:")
    print(f"   Original average difference: +8.9%")
    print(f"   Improved average difference: {avg_diff_improved:+.1f}%")
    print(f"   Improvement: {8.9 - abs(avg_diff_improved):.1f}% points")
    print(f"   Maximum difference: {max_diff_improved:.1f}%")
    
    # Count blended vs model-only forecasts
    blended_count = len([f for f in quarterly_forecasts_improved if f.get('method') == 'BLENDED'])
    model_only_count = len([f for f in quarterly_forecasts_improved if f.get('method') == 'MODEL_ONLY'])
    
    print(f"\n🔧 METHOD BREAKDOWN:")
    print(f"   Blended forecasts (model + ICE): {blended_count}")
    print(f"   Model-only forecasts: {model_only_count}")


📊 COMPARING IMPROVED FORECASTS WITH ICE FUTURES...
🔍 Q1 2026: Model $15.36 vs ICE $14.97 (2.6% ABOVE)
🔍 Q2 2026: Model $15.16 vs ICE $14.48 (4.7% ABOVE)
🔍 Q3 2026: Model $15.20 vs ICE $14.57 (4.3% ABOVE)
🔍 Q1 2027: Model $15.41 vs ICE $15.09 (2.1% ABOVE)
🔍 Q2 2027: Model $15.30 vs ICE $14.82 (3.2% ABOVE)
🔍 Q3 2027: Model $15.32 vs ICE $14.88 (3.0% ABOVE)
✅ Compared 6 improved forecasts

📈 IMPROVEMENT SUMMARY:
   Original average difference: +8.9%
   Improved average difference: +3.3%
   Improvement: 5.6% points
   Maximum difference: 4.7%

🔧 METHOD BREAKDOWN:
   Blended forecasts (model + ICE): 6
   Model-only forecasts: 2


In [21]:
# ======================================================
# 🔹 EXPORT RESULTS TO JSON
# ======================================================

print("\n💾 EXPORTING RESULTS TO JSON...")

def export_forecast_results(quarterly_forecasts, futures_comparison, model_history, output_path="sugar_forecasts_2026_2027.json"):
    """Export all forecast results as JSON"""
    
    # Prepare export data
    export_data = {
        'generation_date': str(pd.Timestamp.now()),
        'data_sources': {
            'last_historical_date': str(df_enhanced_fixed.index[-1]),
            'last_historical_price': float(df_enhanced_fixed['SUGAR'].iloc[-1]),
            'ice_futures_loaded': len(ice_futures) if ice_futures is not None else 0,
            'price_scale': 'dollars_per_pound'
        },
        'model_performance': {
            'final_training_loss': float(history.history['loss'][-1]),
            'final_validation_loss': float(history.history['val_loss'][-1]),
            'final_training_mae': float(history.history['mae'][-1]),
            'final_validation_mae': float(history.history['val_mae'][-1]),
            'training_epochs': len(history.history['loss'])
        },
        'quarterly_forecasts': quarterly_forecasts,
        'market_analysis': {
            'total_forecasts_generated': len(quarterly_forecasts),
            'forecast_price_range': {
                'min': float(min([f['forecast_price'] for f in quarterly_forecasts])),
                'max': float(max([f['forecast_price'] for f in quarterly_forecasts])),
                'average': float(np.mean([f['forecast_price'] for f in quarterly_forecasts]))
            },
            'confidence_summary': {
                'average_confidence': float(np.mean([f['confidence'] for f in quarterly_forecasts])),
                'min_confidence': float(min([f['confidence'] for f in quarterly_forecasts])),
                'max_confidence': float(max([f['confidence'] for f in quarterly_forecasts]))
            }
        }
    }
    
    # Add futures comparison if available
    if futures_comparison is not None and len(futures_comparison) > 0:
        export_data['futures_comparison'] = {
            'total_comparisons': len(futures_comparison),
            'price_difference_summary': {
                'average_difference_pct': float(futures_comparison['price_difference_pct'].mean()),
                'max_positive_difference_pct': float(futures_comparison['price_difference_pct'].max()),
                'max_negative_difference_pct': float(futures_comparison['price_difference_pct'].min()),
                'average_absolute_difference_pct': float(futures_comparison['price_difference_pct'].abs().mean())
            },
            'detailed_comparisons': futures_comparison.to_dict('records')
        }
        
        # Add trading insights
        above_ice = len(futures_comparison[futures_comparison['price_difference_pct'] > 0])
        below_ice = len(futures_comparison[futures_comparison['price_difference_pct'] < 0])
        
        export_data['trading_insights'] = {
            'model_above_ice_count': above_ice,
            'model_below_ice_count': below_ice,
            'model_above_ice_percentage': float(above_ice / len(futures_comparison) * 100),
            'arbitrage_opportunities': [
                {
                    'quarter': row['comparison_quarter'],
                    'model_price': row['model_forecast'],
                    'ice_price': row['ice_futures'],
                    'difference_pct': row['price_difference_pct'],
                    'action': 'BUY_ICE_SELL_MODEL' if row['price_difference_pct'] > 5 else 
                             'BUY_MODEL_SELL_ICE' if row['price_difference_pct'] < -5 else 'HOLD'
                }
                for _, row in futures_comparison.iterrows()
            ]
        }
    
    # Save to JSON file
    with open(output_path, "w") as f:
        json.dump(export_data, f, indent=2, default=str)
    
    print(f"✅ Forecast results exported to {output_path}")
    
    # Also create CSV files for easy analysis
    try:
        # Quarterly forecasts CSV
        forecasts_df = pd.DataFrame(quarterly_forecasts)
        forecasts_df.to_csv("quarterly_forecasts_detailed.csv", index=False)
        print("✅ CSV: quarterly_forecasts_detailed.csv")
        
        # Futures comparison CSV
        if futures_comparison is not None and len(futures_comparison) > 0:
            futures_comparison.to_csv("futures_comparison_detailed.csv", index=False)
            print("✅ CSV: futures_comparison_detailed.csv")
            
        # Summary CSV
        summary_data = []
        for forecast in quarterly_forecasts:
            summary_row = {
                'quarter': forecast['quarter'],
                'date': forecast['date'],
                'model_forecast': forecast['forecast_price'],
                'confidence': forecast['confidence'],
                'days_ahead': forecast['days_ahead']
            }
            
            # Add ICE comparison if available
            if futures_comparison is not None:
                matching_ice = futures_comparison[futures_comparison['comparison_quarter'] == forecast['quarter']]
                if len(matching_ice) > 0:
                    summary_row['ice_futures'] = matching_ice.iloc[0]['ice_futures']
                    summary_row['price_difference_pct'] = matching_ice.iloc[0]['price_difference_pct']
            
            summary_data.append(summary_row)
        
        summary_df = pd.DataFrame(summary_data)
        summary_df.to_csv("forecast_summary.csv", index=False)
        print("✅ CSV: forecast_summary.csv")
        
    except Exception as e:
        print(f"⚠️ Could not create CSV files: {e}")
    
    return export_data

# Export the results
export_results = export_forecast_results(
    quarterly_forecasts_complete, 
    futures_comparison_improved, 
    history
)

print(f"\n📊 EXPORT SUMMARY:")
print(f"   • JSON file: sugar_forecasts_2026_2027.json")
print(f"   • CSV files: quarterly_forecasts_detailed.csv, futures_comparison_detailed.csv, forecast_summary.csv")
print(f"   • Forecasts: {len(quarterly_forecasts_complete)} quarterly_forecasts_complete")
print(f"   • Comparisons: {len(futures_comparison_improved) if futures_comparison_improved is not None else 0} ICE futures comparisons")


💾 EXPORTING RESULTS TO JSON...
✅ Forecast results exported to sugar_forecasts_2026_2027.json
✅ CSV: quarterly_forecasts_detailed.csv
✅ CSV: futures_comparison_detailed.csv
✅ CSV: forecast_summary.csv

📊 EXPORT SUMMARY:
   • JSON file: sugar_forecasts_2026_2027.json
   • CSV files: quarterly_forecasts_detailed.csv, futures_comparison_detailed.csv, forecast_summary.csv
   • Forecasts: 8 quarterly_forecasts_complete
   • Comparisons: 6 ICE futures comparisons


In [22]:
# ======================================================
# 🔹 EXPORT RESULTS TO JSON (INCLUDES ALL QUARTERS)
# ======================================================

print("\n💾 EXPORTING RESULTS TO JSON...")

def export_forecast_results(quarterly_forecasts_data, futures_comparison_data, output_path="sugar_forecasts_2026_2027.json"):
    """Export all forecast results as JSON - includes ALL quarters"""
    
    # Prepare export data
    export_data = {
        'generation_date': str(pd.Timestamp.now()),
        'data_sources': {
            'last_historical_date': str(df_enhanced_fixed.index[-1]),
            'last_historical_price': float(df_enhanced_fixed['SUGAR'].iloc[-1]),
            'ice_futures_loaded': len(ice_futures) if ice_futures is not None else 0,
            'price_scale': 'dollars_per_pound'
        },
        'model_performance': {
            'final_training_loss': float(history.history['loss'][-1]),
            'final_validation_loss': float(history.history['val_loss'][-1]),
            'final_training_mae': float(history.history['mae'][-1]),
            'final_validation_mae': float(history.history['val_mae'][-1]),
            'training_epochs': len(history.history['loss'])
        },
        'quarterly_forecasts': quarterly_forecasts_data,
        'market_analysis': {
            'total_forecasts_generated': len(quarterly_forecasts_data),
            'forecast_price_range': {
                'min': float(min([f['forecast_price'] for f in quarterly_forecasts_data])),
                'max': float(max([f['forecast_price'] for f in quarterly_forecasts_data])),
                'average': float(np.mean([f['forecast_price'] for f in quarterly_forecasts_data]))
            },
            'confidence_summary': {
                'average_confidence': float(np.mean([f['confidence'] for f in quarterly_forecasts_data])),
                'min_confidence': float(min([f['confidence'] for f in quarterly_forecasts_data])),
                'max_confidence': float(max([f['confidence'] for f in quarterly_forecasts_data]))
            }
        }
    }
    
    # Create comprehensive comparison that includes ALL quarters
    all_comparisons = []
    
    for forecast in quarterly_forecasts_data:
        comparison_entry = {
            'quarter': forecast['quarter'],
            'model_price': forecast['forecast_price'],
            'confidence': forecast['confidence'],
            'days_ahead': forecast['days_ahead']
        }
        
        # Try to find matching ICE futures
        ice_match = None
        if futures_comparison_data is not None:
            matching_ice = futures_comparison_data[futures_comparison_data['comparison_quarter'] == forecast['quarter']]
            if len(matching_ice) > 0:
                ice_match = matching_ice.iloc[0]
        
        if ice_match is not None:
            # Has ICE futures comparison
            comparison_entry['ice_price'] = ice_match['ice_futures']
            comparison_entry['difference_pct'] = ice_match['price_difference_pct']
            comparison_entry['ice_data_available'] = True
            # Use the same action logic
            diff_pct = ice_match['price_difference_pct']
            comparison_entry['action'] = 'BUY_ICE_SELL_MODEL' if diff_pct > 5 else 'BUY_MODEL_SELL_ICE' if diff_pct < -5 else 'HOLD'
        else:
            # No ICE futures available for this quarter
            comparison_entry['ice_price'] = None
            comparison_entry['difference_pct'] = None
            comparison_entry['ice_data_available'] = False
            comparison_entry['action'] = 'MODEL_ONLY_NO_ICE'
        
        all_comparisons.append(comparison_entry)
    
    # Add the comprehensive comparison to export data
    export_data['comprehensive_comparison'] = all_comparisons
    
    # Also keep the original futures comparison for reference
    if futures_comparison_data is not None and len(futures_comparison_data) > 0:
        export_data['futures_comparison'] = {
            'total_comparisons': len(futures_comparison_data),
            'price_difference_summary': {
                'average_difference_pct': float(futures_comparison_data['price_difference_pct'].mean()),
                'max_positive_difference_pct': float(futures_comparison_data['price_difference_pct'].max()),
                'max_negative_difference_pct': float(futures_comparison_data['price_difference_pct'].min()),
                'average_absolute_difference_pct': float(futures_comparison_data['price_difference_pct'].abs().mean())
            },
            'detailed_comparisons': futures_comparison_data.to_dict('records')
        }
    
    # Add trading insights based on comprehensive comparison
    comparisons_with_ice = [c for c in all_comparisons if c['ice_data_available']]
    if comparisons_with_ice:
        above_ice = len([c for c in comparisons_with_ice if c.get('difference_pct', 0) > 0])
        below_ice = len([c for c in comparisons_with_ice if c.get('difference_pct', 0) < 0])
        
        export_data['trading_insights'] = {
            'total_quarters_with_ice_data': len(comparisons_with_ice),
            'total_quarters_model_only': len([c for c in all_comparisons if not c['ice_data_available']]),
            'model_above_ice_count': above_ice,
            'model_below_ice_count': below_ice,
            'model_above_ice_percentage': float(above_ice / len(comparisons_with_ice) * 100) if comparisons_with_ice else 0,
            'arbitrage_opportunities': [
                {
                    'quarter': comp['quarter'],
                    'model_price': comp['model_price'],
                    'ice_price': comp['ice_price'],
                    'difference_pct': comp['difference_pct'],
                    'action': comp['action']
                }
                for comp in comparisons_with_ice if comp['action'] != 'HOLD'
            ]
        }
    
    # Save to JSON file
    with open(output_path, "w") as f:
        json.dump(export_data, f, indent=2, default=str)
    
    print(f"✅ Forecast results exported to {output_path}")
    
    # Also create CSV files for easy analysis
    try:
        # Quarterly forecasts CSV
        forecasts_df = pd.DataFrame(quarterly_forecasts_data)
        forecasts_df.to_csv("quarterly_forecasts_detailed.csv", index=False)
        print("✅ CSV: quarterly_forecasts_detailed.csv")
        
        # Comprehensive comparison CSV
        comp_df = pd.DataFrame(all_comparisons)
        comp_df.to_csv("comprehensive_comparison.csv", index=False)
        print("✅ CSV: comprehensive_comparison.csv")
        
        # Summary CSV
        summary_data = []
        for forecast in quarterly_forecasts_data:
            summary_row = {
                'quarter': forecast['quarter'],
                'date': forecast['date'],
                'model_forecast': forecast['forecast_price'],
                'confidence': forecast['confidence'],
                'days_ahead': forecast['days_ahead']
            }
            
            # Add ICE comparison if available
            matching_comp = next((c for c in all_comparisons if c['quarter'] == forecast['quarter']), None)
            if matching_comp and matching_comp['ice_data_available']:
                summary_row['ice_futures'] = matching_comp['ice_price']
                summary_row['price_difference_pct'] = matching_comp['difference_pct']
                summary_row['action'] = matching_comp['action']
            else:
                summary_row['ice_futures'] = None
                summary_row['price_difference_pct'] = None
                summary_row['action'] = 'MODEL_ONLY'
            
            summary_data.append(summary_row)
        
        summary_df = pd.DataFrame(summary_data)
        summary_df.to_csv("forecast_summary.csv", index=False)
        print("✅ CSV: forecast_summary.csv")
        
    except Exception as e:
        print(f"⚠️ Could not create CSV files: {e}")
    
    return export_data

# Choose which forecasts to export
if 'quarterly_forecasts_improved' in locals() and len(quarterly_forecasts_improved) > 0:
    forecasts_to_export = quarterly_forecasts_improved
    comparison_to_export = futures_comparison_improved if 'futures_comparison_improved' in locals() else None
    print("📊 Using IMPROVED forecasts for export")
elif 'quarterly_forecasts_complete' in locals() and len(quarterly_forecasts_complete) > 0:
    forecasts_to_export = quarterly_forecasts_complete
    comparison_to_export = futures_comparison if 'futures_comparison' in locals() else None
    print("📊 Using COMPLETE forecasts for export")
else:
    print("❌ No forecast data found for export")
    forecasts_to_export = []
    comparison_to_export = None

# Export the results
if len(forecasts_to_export) > 0:
    export_results = export_forecast_results(
        forecasts_to_export, 
        comparison_to_export
    )

    print(f"\n📊 EXPORT SUMMARY:")
    print(f"   • JSON file: sugar_forecasts_2026_2027.json")
    print(f"   • Forecasts: {len(forecasts_to_export)} quarterly forecasts")
    print(f"   • Quarters with ICE data: {len([f for f in export_results['comprehensive_comparison'] if f['ice_data_available']])}")
    print(f"   • Model-only quarters: {len([f for f in export_results['comprehensive_comparison'] if not f['ice_data_available']])}")
    print(f"   • Average confidence: {np.mean([f['confidence'] for f in forecasts_to_export]):.1%}")
else:
    print("❌ No data available for export")


💾 EXPORTING RESULTS TO JSON...
📊 Using IMPROVED forecasts for export
✅ Forecast results exported to sugar_forecasts_2026_2027.json
✅ CSV: quarterly_forecasts_detailed.csv
✅ CSV: comprehensive_comparison.csv
✅ CSV: forecast_summary.csv

📊 EXPORT SUMMARY:
   • JSON file: sugar_forecasts_2026_2027.json
   • Forecasts: 8 quarterly forecasts
   • Quarters with ICE data: 6
   • Model-only quarters: 2
   • Average confidence: 44.3%
